In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
csv = '../Preprocessing/ImputedSets/bronx_imputed.csv'
df = pd.DataFrame(pd.read_csv(csv))

In [4]:
df.head()

,Borough,Area,Address,Num_Beds,Num_Baths,Sq-Ft,Rent
0,bronx,Tremont,2065 Ryer Avenue,2.0,1.0,719.00,2217.0
1,bronx,Crotona Park East,1169 Hoe Avenue,2.0,1.0,848.76,2425.0
2,bronx,Highbridge,1515 Macombs Road,1.0,1.0,631.43,1800.0
3,bronx,Norwood,31 West Mosholu Parkway North,2.0,1.0,833.22,1950.0
4,bronx,New_Dev,2401 3rd Avenue,1.0,1.0,675.00,3132.0
